# Creating Synthetic Experts with Generative AI
> ## Prediction with Fine-Tuned Model  
*QuickStart* for individual Tweets
  
Version 1.0   
Date: September 2, 2023    
Author: Daniel M. Ringel    
Contact: dmr@unc.edu

*Daniel M. Ringel, Creating Synthetic Experts with Generative Artificial Intelligence (July 15, 2023).  
Available at SSRN: https://papers.ssrn.com/abstract_id=4542949*

#### Requirements
- PyTorch
- BeautifulSoup
- Huggingfaces transformers
- Warnings and regular expressions (re)

##### Apple M1/M2 GPU MPS Requirements (Optional)
> See Python Notebook: [Setup-MacBook-M2-Pytorch-TensorFlow-Apr2023.ipynb](https://github.com/dringel/Synthetic-Experts)  

- Mac computer with Apple silicon GPU
- macOS 12.3 or later
- Python 3.7 or later
- Xcode command-line tools: xcode-select --install

##### If you have no GPU available, the code falls back to your CPU

In [1]:
# Imports
import pandas as pd, numpy as np, warnings, torch, re
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from bs4 import BeautifulSoup
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

In [2]:
# Helper Functions
def clean_and_parse_tweet(tweet):
    tweet = re.sub(r"https?://\S+|www\.\S+", " URL ", tweet)
    parsed = BeautifulSoup(tweet, "html.parser").get_text() if "filename" not in str(BeautifulSoup(tweet, "html.parser")) else None
    return re.sub(r" +", " ", re.sub(r'^[.:]+', '', re.sub(r"\\n+|\n+", " ", parsed or tweet)).strip()) if parsed else None

def predict_tweet(tweet, model, tokenizer, device, threshold=0.5):
    inputs = tokenizer(tweet, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    probs = torch.sigmoid(model(**inputs).logits).detach().cpu().numpy()[0]
    return probs, [id2label[i] for i, p in enumerate(probs) if id2label[i] in {'Product', 'Place', 'Price', 'Promotion'} and p >= threshold]

In [3]:
# Setup
device = "mps" if "backends" in dir(torch) and hasattr(torch.backends, 'mps') and torch.backends.mps.is_built() and torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
synxp = "dmr76/mmx_classifier_microblog_ENv02"
model = AutoModelForSequenceClassification.from_pretrained(synxp).to(device)
tokenizer = AutoTokenizer.from_pretrained(synxp)
id2label = model.config.id2label

In [4]:
# ---->>> Define your Tweet  <<<----
tweet = "Best cushioning ever!!! 🤗🤗🤗  my zoom vomeros are the bomb🏃🏽‍♀️💨!!!  \n @nike #run #training https://randomurl.ai"

In [5]:
# Clean and Predict
cleaned_tweet = clean_and_parse_tweet(tweet)
probs, labels = predict_tweet(cleaned_tweet, model, tokenizer, device)

# Print Labels and Probabilities
print(labels, probs)

['Product'] [0.9948836  0.01391613 0.00593502 0.01573174]
